In [14]:
#to classfy a sms intp spam and ham using multinominal naives bayes theorem
import pandas as pd
url = 'https://raw.githubusercontent.com/justmarkham/pycon-2016-tutorial/master/data/sms.tsv'
sms = pd.read_table(url, header=None, names=['label', 'message'])

In [15]:
print(sms.shape)
print("\n")
print(sms['label'].value_counts())
sms

(5572, 2)


ham     4825
spam     747
Name: label, dtype: int64


,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [16]:
# convert label to a numerical variable
sms['label_num']=sms['label'].map({'ham':0,'spam':1})
sms

,label,message,label_num
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0
...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,1
5568,ham,Will ü b going to esplanade fr home?,0
5569,ham,"Pity, * was in mood for that. So...any other s...",0
5570,ham,The guy did some bitching but I acted like i'd...,0


In [18]:
# how to define X and y (from the SMS data) for use with COUNTVECTORIZER
X=sms.message
y=sms.label_num

In [20]:
# split X and y into training and testing sets
# by default, it splits 75% training and 25% test

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=1)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(4179,)
(1393,)
(4179,)
(1393,)


In [22]:
#Vectorizing our dataset
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer()
X_train_dtm = cv.fit_transform(X_train)

#transform testing data (using fitted vocabulary) into a document-term matrix
X_test_dtm=cv.transform(X_test)

The multinomial Naive Bayes classifier is suitable for classification with discrete features (e.g., word counts for text classification). The multinomial distribution normally requires integer feature counts. However, in practice, fractional counts such as tf-idf may also work.

In [26]:
#building and evaluating model
from sklearn.naive_bayes import MultinomialNB
nb=MultinomialNB()
# train the model using X_train_dtm 
%time nb.fit(X_train_dtm,y_train)

CPU times: user 5.79 ms, sys: 89 µs, total: 5.88 ms
Wall time: 7.13 ms


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [28]:
#make prediction on x_test
y_pred_class = nb.predict(X_test_dtm)

from sklearn import metrics
metrics.accuracy_score(y_test,y_pred_class)

# print the confusion matrix
metrics.confusion_matrix(y_test,y_pred_class)

"""
Confusion matrix
[TN FP
FN TP]
"""

array([[1203,    5],
       [  11,  174]])

In [29]:
# print message text for the false positives (ham incorrectly classified as spam)

X_test[y_pred_class > y_test]

574               Waiting for your call.
3375             Also andros ice etc etc
45      No calls..messages..missed calls
3415             No pic. Please re-send.
1988    No calls..messages..missed calls
Name: message, dtype: object

In [30]:
# print message text for the false negatives (spam incorrectly classified as ham)

X_test[y_pred_class < y_test]

3132    LookAtMe!: Thanks for your purchase of a video...
5       FreeMsg Hey there darling it's been 3 week's n...
3530    Xmas & New Years Eve tickets are now on sale f...
684     Hi I'm sue. I am 20 years old and work as a la...
1875    Would you like to see my XXX pics they are so ...
1893    CALL 09090900040 & LISTEN TO EXTREME DIRTY LIV...
4298    thesmszone.com lets you send free anonymous an...
4949    Hi this is Amy, we will be sending you a free ...
2821    INTERFLORA - It's not too late to order Inter...
2247    Hi ya babe x u 4goten bout me?' scammers getti...
4514    Money i have won wining number 946 wot do i do...
Name: message, dtype: object

In [32]:
# calculate AUC
y_pred_prob = nb.predict_proba(X_test_dtm)[:, 1]
metrics.roc_auc_score(y_test, y_pred_prob)

#AUC is useful as a single number summary of classifier performance
#Higher value = better classifier

0.9866431000536962